In [16]:
## import sys
import jsonpickle
import os
import tweepy
from tweepy import OAuthHandler
import pytz
from pytz import timezone
from tweepy import OAuthHandler
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string
from nltk import trigrams
import re
from uber_rides.session import Session
from uber_rides.client import UberRidesClient
import json
import math
import pandas as pd
from pandas.io.json import json_normalize



consumer_key = 'gswlPdHuI7e0qWBAT5OavoH1w'
consumer_secret = 'rvWMe0ldojIsYSUISbcqKxFodIl0VeMGSLf8HOWUcSL6qOywgK'
access_token = '27651816-os3dMfworD2o430p66jdr668XVzvAOnz16YZgAtfj'
access_secret = 'cE8cV46N6qrKzWfuIVogYPaeM003hd2VH6Y38S4T39YGs'
API_KEY = 'gswlPdHuI7e0qWBAT5OavoH1w'
API_SECRET = 'rvWMe0ldojIsYSUISbcqKxFodIl0VeMGSLf8HOWUcSL6qOywgK'

uber_session = Session(server_token='-l3XEhlZRPidhD3pbwMAT44BAn6W-9ZdWqlV9EJU')
uber_client = UberRidesClient(uber_session)


dx = 15000
dy = 15000

def GetUBerEstimate(lat_origin, long_origin):
    lat_dest = lat_origin + (180/math.pi)*(dy/6378137)
    long_dest = long_origin + (180/math.pi)*(dx/6378137)/math.cos(lat_origin)
    
    response = uber_client.get_price_estimates(
    start_latitude=lat_origin,
    start_longitude=long_origin,
    end_latitude=lat_dest,
    end_longitude=long_dest,
    seat_count=2
    )
    
    estimate = response.json.get('prices')
    high_estimate_uberx = estimate[1]['high_estimate']
    return str(high_estimate_uberx)


auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
    
searchQuery = '#traffic'  # this is what we're searching for
maxTweets = 50000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets_dataset_1.txt' # We'll store the tweets in a text file.





eastern = timezone('US/Eastern')
utc = timezone('UTC')


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
#max_Id_to_start_twitter_search_from
max_id = 829863674068205569

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    #f.write("tweet_id" + '\t' + "tweet_time" + '\t' + "tweet_text" +'\t' + "tweet_coordinates" + '\t' + "lat_origin" + '\t' + "long_origin" + '\t' + "uber_estimate" + '\n')
    f.write("tweet_id\ttweet_time\ttweet_text\ttweet_coordinates\tlat_origin\tlong_origin\tuber_estimate\n")
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                if (tweet.geo != None):
                    tweet_text=tweet.text
                    tweet_id=jsonpickle.encode(tweet.id, unpicklable=False)
                    created_at=jsonpickle.encode(tweet.created_at, unpicklable=False)
                    utc_created_at = utc.localize(tweet.created_at).astimezone(eastern)
                    tweet_text = jsonpickle.encode(tweet.text, unpicklable=False)
                    tweet_coordinates= jsonpickle.encode(tweet.geo['coordinates'],unpicklable=False)
                    lat_origin=(tweet_coordinates.split(",",1)[0].split("[",1)[1]).strip()
                    long_origin=(tweet_coordinates.split(",",1)[1].split("]",1)[0]).strip()
                    uber_estimate = GetUBerEstimate(float(lat_origin),float(long_origin))
                    f.write(tweet_id + '\t' + str(utc_created_at) + '\t' + tweet_text +'\t' + tweet_coordinates + '\t'+lat_origin+'\t' +long_origin+'\t' + uber_estimate + '\n')
            tweetCount += len(new_tweets)
            if tweetCount % 1000 == 0:
                print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))


IndexError: list index out of range